<a href="https://colab.research.google.com/github/Jaime44/WorkSpace/blob/main/models/colaborative%20filter/00_FC_Recommender_System_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/TFM/WorkSpace/Models/colaborative filter'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = os.getcwd().replace("\\", "/")

datasets_path = "/datasets/"
path_absolute = path_absolute+datasets_path

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate, train_test_split

In [ ]:
#Carga de datos y preprocesado

dataFrame = pd.read_csv(path_absolute+'/df_mezclado_tags_ratings_movies_links_genTags.csv')

print(dataFrame.shape)

data = dataFrame.copy()
# data = data.sample(n=10000, random_state=42)

# Inserta la columna 'rating' en la última posición del DataFrame
data.insert(len(data.columns)-1, 'rating', data.pop('rating'))
# Inserta la columna 'relevance' en la penúltima posición del DataFrame
data.insert(len(data.columns)-2, 'relevance', data.pop('relevance'))
# Inserta la columna 'tag_etiquetas_genómicas' despues de tag_df_mezclado
data.insert(5, 'tag_etiquetas_genómicas', data.pop('tag_etiquetas_genómicas'))

# Renombrar algunas columnas:
data.rename(columns={'timestamp_valoraciones': 'timestamp_rt', 'timestamp_etiquetas': 'timestamp_tags', 'tag_df_mezclado_tags_ratings_movies_links_genMov': 'tag_by_user', 'tag_etiquetas_genómicas': 'gen_tag'}, inplace=True)

# Elimino las columnas:
columnas_a_eliminar = ['imdbId', 'tmdbId']
data = data.drop(columnas_a_eliminar, axis=1)
print(data.shape)
data.head()

df = data.copy()

In [ ]:
# Se cuenta los valores unicos de usuarios
df['userId'].value_counts()

In [ ]:
# Se cuenta los valores unicos de rating
df['rating'].value_counts()


In [ ]:
rango_valores = df['rating'].describe()['min'], df['rating'].describe()['max']
print("Rango de valores cd rating:", rango_valores)

In [ ]:
# Se cuenta los valores unicos de peliculas
df['movieId'].value_counts()

In [ ]:
#promedio de calificaciones agrupado por titulos
df.groupby('title')['rating'].mean().sort_values(ascending=False).head(10)

In [ ]:
#hisotgrama de la distribución de las disitntas calificaciones
df['rating'].hist(figsize= (10,10))
plt.show()

### Analizar la distribución del número de votos (o calificaciones) dados por los usuarios a las películas y el número de películas calificadas por cada usuario en un conjunto de datos de películas.

In [ ]:
dataFrame = data.copy()

In [ ]:
# Contar el número de votos (calificaciones) por cada película
numberOf_user_voted_for_movie = pd.DataFrame(dataFrame.groupby('movieId')['rating'].agg('count'))
numberOf_user_voted_for_movie.reset_index(level = 0,inplace = True)
numberOf_user_voted_for_movie.head()

In [ ]:
# Descripción estadística del número de votos por película
numberOf_user_voted_for_movie.describe()

La tabla proporciona estadísticas descriptivas sobre el número de votos por película (rating) en el conjunto de datos. Aquí está la interpretación de las estadísticas:

1. count (Recuento):
    Hay 12,105 películas en el conjunto de datos.
    
2. mean (Media):
    La media del número de votos por película es aproximadamente 739.31.

3. std (Desviación estándar):
    La desviación estándar es alta, aproximadamente 2308.42. Esto indica que la variabilidad en el número de votos por película es significativa y que hay películas con un rango amplio de votos.

4. min (Mínimo):
    La película con el menor número de votos tiene 3 votos.

5. 25% (Cuartil 1 - Q1):
    El 25% de las películas tienen 56 votos o menos.

6. 50% (Cuartil 2 - Mediana):
    La mediana (o el percentil 50) es 160, lo que significa que el 50% de las películas tienen 160 votos o menos.

7. 75% (Cuartil 3 - Q3):
    El 75% de las películas tienen 490 votos o menos.

8. max (Máximo):
    La película con el mayor número de votos tiene 84,546 votos.
    
En resumen, la distribución del número de votos por película varía significativamente, con algunas películas que tienen un gran número de votos, mientras que otras tienen menos votos. La mediana proporciona una indicación de la tendencia central, y la alta desviación estándar sugiere que la dispersión es amplia, lo que significa que algunas películas tienen una cantidad significativamente mayor de votos que otras.

In [ ]:
# Se cuenta el número de películas calificadas por cada usuario.
numberOf_movies_voted_by_user = pd.DataFrame(dataFrame.groupby('userId')['rating'].agg('count'))
numberOf_movies_voted_by_user.reset_index(level = 0,inplace = True)
numberOf_movies_voted_by_user.head()

In [ ]:
# Descripción estadística del número de películas calificadas por usuario
numberOf_movies_voted_by_user.describe()

La tabla proporciona estadísticas descriptivas sobre el número de películas calificadas (rating) por usuario en el conjunto de datos. Aquí está la interpretación de las estadísticas:

1. count (Recuento):
    Hay 12,918 usuarios en el conjunto de datos.

2. mean (Media):
    La media del número de películas calificadas por usuario es aproximadamente 692.78.

3. std (Desviación estándar):
    La desviación estándar es alta, aproximadamente 4142.74. Esto indica que la variabilidad en el número de películas calificadas por usuario es significativa y que hay usuarios que han calificado un rango amplio de películas.

4. min (Mínimo):
    El usuario con el menor número de películas calificadas ha calificado al menos 3 películas.

5. 25% (Cuartil 1 - Q1):
    El 25% de los usuarios han calificado 26 películas o menos.

6. 50% (Cuartil 2 - Mediana):
    La mediana (o el percentil 50) es 56, lo que significa que el 50% de los usuarios han calificado 56 películas o menos.

7. 75% (Cuartil 3 - Q3):
    El 75% de los usuarios han calificado 235 películas o menos.

8. max (Máximo):
    El usuario con el mayor número de películas calificadas ha calificado hasta 217,078 películas.

En resumen, la distribución del número de películas calificadas por usuario varía significativamente, con algunos usuarios que han calificado un gran número de películas y otros que han calificado menos. La mediana proporciona una indicación de la tendencia central, y la alta desviación estándar sugiere que la dispersión es amplia, lo que significa que algunos usuarios han calificado muchas más películas que otros.

In [ ]:
# Visualización del número de votos por película
plt.figure()
ax = sns.scatterplot(y = 'rating', x = 'movieId', data = numberOf_user_voted_for_movie)
plt.axhline(y=50,color='r')
plt.ylabel('Número de votos de usuariuos por pelicula')

In [ ]:
# Número de películas calificadas por cada usuario
plt.figure()
ax = sns.scatterplot(y = 'rating', x = 'userId', data = numberOf_movies_voted_by_user)
plt.axhline(y=60,color='r')
plt.ylabel('Número de peliculas calificadas por usuario')

In [ ]:
# Correlación de las diferentes variables usando un mapa de calor.
# Util para identificar patrones y ralciones entre variables.
# 1 (color claro): Correlación perfecta positiva. Aumentos en una variable se asocian
#     con aumentos proporcionales en la otra en una relación lineal perfecta.

# 0 (color intermedio): Falta de correlación. No hay una relación lineal entre las dos variables;
#     cambios en una variable no predicen cambios en la otra.
tc = df.corr()
sns.heatmap(tc)
plt.show()

In [ ]:
#Data frame con la media de calificaciones por pelicula y nímero de calificaciones
ratings =pd.DataFrame(df.groupby('title')['rating'].mean())
ratings['rating_numbers'] = pd.DataFrame(df.groupby('title')['rating'].count())
ratings.head()

In [ ]:
#Creación de una matriz Pivot table
#Matriz que representa las calificaciones dadas por los usuarios a las películas.
# Las filas representan usuarios, las columnas representan títulos de películas y los valores son las calificaciones.
# Las celdas que no tienen calificación se llenan con ceros.
pivot_table_result = df.pivot_table(index='userId', columns='title', values='rating')
pivot_table_result.fillna(0, inplace=True)
pivot_table_result.shape
pivot_table_result.head()

In [ ]:
#Creación de una matriz Pivot table con un umbral de al menos 10 peliuculas calificadas
print("Before: ",pivot_table_result.shape)
pivot_table_umbral_rating = pivot_table_result.copy()
pivot_table_umbral_rating = pivot_table_umbral_rating.dropna(thresh=10, axis=1).fillna(0,axis=1)
print("After: ",pivot_table_umbral_rating.shape)

In [ ]:
# Pivot table sin rellenar 0
pivot_table_nan = df.pivot_table(index='userId', columns='title', values='rating')
pivot_table_nan.shape
pivot_table_nan.head()

Sí, tiene sentido eliminar filas duplicadas antes de crear la tabla pivote, especialmente si estás trabajando con datos de calificaciones de usuarios donde cada fila representa la calificación de un usuario para una película específica. Algunas razones para hacer esto son:

1. **Evitar Duplicados en la Tabla Pivote:**
   - Eliminar duplicados garantiza que no haya varias entradas para la misma combinación de usuario y película al crear la tabla pivote. Esto es crucial para obtener resultados coherentes y evitar distorsiones en las correlaciones.

2. **Precisión y Consistencia:**
   - La eliminación de duplicados mejora la precisión de las correlaciones, ya que cada entrada en la tabla pivote representa de manera única una calificación de un usuario para una película específica. Esto ayuda a evitar que las correlaciones estén influenciadas por datos duplicados.

3. **Eficiencia Computacional:**
   - Al reducir el tamaño del conjunto de datos antes de crear la tabla pivote, puedes mejorar la eficiencia computacional, especialmente si tienes un gran conjunto de datos. Menos duplicados significa menos cálculos y un mejor rendimiento.

La línea de código que proporcionaste, `ratings = ratings.drop_duplicates(subset=['userId', 'movieId'])`, eliminará las filas duplicadas basadas en las columnas 'userId' y 'movieId'. Asegúrate de revisar los resultados después de esta operación para confirmar que los duplicados se han eliminado según tus expectativas.

In [ ]:
# Se implementa el recomendador según una pelicula. Se usa la función de correlación.

def recomendar_peliculas(nombre_pelicula, tabla_pivote, ratings,
                         min_cantidad_calificaciones=50, tipo_correlacion='pearson'):
    """
    Recomienda películas basadas en la correlación de calificaciones de usuarios.

    Parámetros:
    - nombre_pelicula: Nombre de la película para la cual se desean recomendaciones.
    - tabla_pivote: DataFrame que contiene las calificaciones de los usuarios (formato de tabla pivote).
    - ratings: DataFrame que contiene información adicional, incluyendo 'cantidad_calificaciones'.
    - min_cantidad_calificaciones: Cantidad mínima de calificaciones para considerar una película.
    - tipo_correlacion: Tipo de correlación a utilizar (pearson, kendall, spearman).

    Retorna:
    - DataFrame de películas recomendadas con puntajes de correlación.
    """

    # Extraer las calificaciones de usuarios para la película especificada
    calificaciones_usuario = tabla_pivote[nombre_pelicula]

    # Calcular correlaciones con otras películas.
    # Usando correlación de Pearson default
    # Compara las calificaciones de la película de entrada con las
    # calificaciones de cada otra película en términos de su similitud
    peliculas_similares = tabla_pivote.corrwith(calificaciones_usuario, method=tipo_correlacion)

    # Crear un DataFrame con los puntajes de correlación
    peliculas_corr = pd.DataFrame(peliculas_similares, columns=['Correlacion'])

    # Eliminar películas sin datos de correlación
    peliculas_corr.dropna(inplace=True)

    # Unir con el DataFrame de calificaciones para considerar la cantidad mínima de calificaciones
    peliculas_corr = peliculas_corr.join(ratings['rating_numbers'], how='left')

    # Filtrar películas según la cantidad mínima de calificaciones y ordenar por correlación
    peliculas_recomendadas = peliculas_corr[peliculas_corr['rating_numbers'] > min_cantidad_calificaciones].sort_values('Correlacion', ascending=False)

    return peliculas_recomendadas


In [ ]:
# Ejemplo usando correlación de Pearson
recomendaciones_pearson_ptr = recomendar_peliculas('Harry Potter and the Goblet of Fire (2005)', pivot_table_result,
                                               ratings, min_cantidad_calificaciones=50, tipo_correlacion='pearson')

# Ejemplo usando correlación de Kendall
recomendaciones_kendall_ptr = recomendar_peliculas('Harry Potter and the Goblet of Fire (2005)', pivot_table_result,
                                               ratings, min_cantidad_calificaciones=50, tipo_correlacion='kendall')

# Ejemplo usando correlación de Spearman
recomendaciones_spearman_ptr = recomendar_peliculas('Harry Potter and the Goblet of Fire (2005)', pivot_table_result,
                                                ratings, min_cantidad_calificaciones=50, tipo_correlacion='spearman')


In [ ]:
## Prueba con la matriz con el umbral de calificaciones por pelicula

# Ejemplo usando correlación de Pearson
recomendaciones_pearson_ptur = recomendar_peliculas('Harry Potter and the Goblet of Fire (2005)', pivot_table_umbral_rating,
                                               ratings, min_cantidad_calificaciones=50, tipo_correlacion='pearson')
print("Peliculas recomendadas habiendo visto Harry Potter and the Goblet of Fire (2005). pearson: ")
recomendaciones_pearson_ptur.head(10)

# Ejemplo usando correlación de Kendall
recomendaciones_kendall_ptur = recomendar_peliculas('Harry Potter and the Goblet of Fire (2005)', pivot_table_umbral_rating,
                                               ratings, min_cantidad_calificaciones=50, tipo_correlacion='kendall')

print("Peliculas recomendadas habiendo visto Harry Potter and the Goblet of Fire (2005). kendall: ")
recomendaciones_kendall_ptur.head(10)

# Ejemplo usando correlación de Spearman
recomendaciones_spearman_ptur = recomendar_peliculas('Harry Potter and the Goblet of Fire (2005)', pivot_table_umbral_rating,
                                                ratings, min_cantidad_calificaciones=50, tipo_correlacion='spearman')
print("Peliculas recomendadas habiendo visto Harry Potter and the Goblet of Fire (2005). spearman: ")
recomendaciones_spearman_ptur.head(10)

In [ ]:
# Prueba con la matriz sin los 0
recomendaciones_pearson_nan = recomendar_peliculas('Harry Potter and the Goblet of Fire (2005)',
                                                   pivot_table_nan, ratings,
                                                   min_cantidad_calificaciones=50, tipo_correlacion='pearson')


print("Peliculas recomendadas habiendo visto Harry Potter and the Goblet of Fire (2005). Matriz con valores NaN: ")
recomendaciones_pearson_nan.head(10)


In [ ]:
import pandas as pd

def recomendar_peliculas(usuario_id, tabla_pivote, ratings, min_cantidad_calificaciones=50,
                         top_n=10, tipo_correlacion='pearson'):
    """
    Recomienda películas no vistas por un usuario basándose en la correlación de calificaciones de usuarios.

    Parámetros:
    - usuario_id: ID del usuario para el cual se desean recomendaciones.
    - tabla_pivote: DataFrame que contiene las calificaciones de los usuarios (formato de tabla pivote).
    - ratings: DataFrame que contiene información adicional, incluyendo 'cantidad_calificaciones'.
    - min_cantidad_calificaciones: Cantidad mínima de calificaciones para considerar una película.
    - top_n: Número de películas recomendadas a devolver.
    - tipo_correlacion: Tipo de correlación a utilizar (pearson, kendall, spearman).

    Retorna:
    - DataFrame de películas no vistas recomendadas con puntajes de correlación.
    """

    # Extraer las calificaciones del usuario
    calificaciones_usuario = tabla_pivote.loc[usuario_id]

    # Filtrar películas no vistas por el usuario
    peliculas_no_vistas = calificaciones_usuario[calificaciones_usuario.isna()].index

    # Calcular correlaciones con las películas no vistas. Usando el tipo de correlación especificado
    # Mide la correlación lineal entre dos variables continuas. Toma valores entre -1 y 1, donde 1
    # significa una correlación positiva perfecta, -1 significa una correlación negativa perfecta, y
    # 0 significa ausencia de correlación lineal.
    peliculas_similares = tabla_pivote[peliculas_no_vistas].corrwith(calificaciones_usuario, method=tipo_correlacion)

    # Crear un DataFrame con los puntajes de correlación
    peliculas_corr = pd.DataFrame(peliculas_similares, columns=['Correlacion'])

    # Eliminar películas sin datos de correlación
    peliculas_corr.dropna(inplace=True)

    # Unir con el DataFrame de calificaciones para considerar la cantidad mínima de calificaciones
    peliculas_corr = peliculas_corr.join(ratings['rating_numbers'], how='left')

    # Filtrar películas según la cantidad mínima de calificaciones y ordenar por correlación
    peliculas_recomendadas = peliculas_corr[peliculas_corr['rating_numbers'] > min_cantidad_calificaciones].sort_values('Correlacion', ascending=False)

    # Tomar las primeras 'top_n' películas recomendadas
    peliculas_recomendadas = peliculas_recomendadas.head(top_n)

    return peliculas_recomendadas


In [ ]:
userID = 264

# Ejemplo usando correlación de Pearson
recomendaciones_pearson = recomendar_peliculas(userID, pivot_table_result, ratings, min_cantidad_calificaciones=50,
                                               top_n=10, tipo_correlacion='pearson')

print(f"Peliculas recomendadas para el usuario {userID}. pearson: ")
recomendaciones_pearson.head(10)

# Ejemplo usando correlación de Kendall
recomendaciones_kendall = recomendar_peliculas(userID, pivot_table_result, ratings, min_cantidad_calificaciones=50,
                                               top_n=10, tipo_correlacion='kendall')

print(f"Peliculas recomendadas para el usuario {userID}. kendall: ")
recomendaciones_kendall.head(10)

# Ejemplo usando correlación de Spearman
recomendaciones_spearman = recomendar_peliculas(userID, pivot_table_result, ratings, min_cantidad_calificaciones=50,
                                                top_n=10, tipo_correlacion='spearman')

print(f"Peliculas recomendadas para el usuario {userID}. spearman: ")
recomendaciones_spearman.head(10)


In [ ]:
## Prueba con la matriz con el umbral de calificaciones por pelicula

# Ejemplo usando correlación de Pearson
recomendaciones_pearson = recomendar_peliculas(userID, pivot_table_umbral_rating, ratings, min_cantidad_calificaciones=50,
                                               top_n=10, tipo_correlacion='pearson')

print(f"Peliculas recomendadas para el usuario {userID}. spearman: ")
recomendaciones_pearson.head(10)

# Ejemplo usando correlación de Kendall
recomendaciones_kendall = recomendar_peliculas(userID, pivot_table_umbral_rating, ratings, min_cantidad_calificaciones=50,
                                               top_n=10, tipo_correlacion='kendall')

print(f"Peliculas recomendadas para el usuario {userID}. spearman: ")
recomendaciones_kendall.head(10)

# Ejemplo usando correlación de Spearman
recomendaciones_spearman = recomendar_peliculas(userID, pivot_table_umbral_rating, ratings, min_cantidad_calificaciones=50,
                                                top_n=10, tipo_correlacion='spearman')

print(f"Peliculas recomendadas para el usuario {userID}. spearman: ")
recomendaciones_spearman.head(10)

-----------------------------------------------------------------------------------------

In [ ]:
# RESET dataframe variable
df = data.copy()
df.head()

In [ ]:
# La línea de código carga un conjunto de datos utilizando la librería surprise
# para modelos de recomendación. Se seleccionan las columnas 'userId', 'movieId', y
# 'rating' del DataFrame ratings. El objeto Reader() se utiliza para especificar cómo
# interpretar las calificaciones, y luego Dataset.load_from_df carga los datos en un
# formato compatible con surprise. Este conjunto de datos surprise puede ser empleado para
# entrenar y evaluar modelos de recomendación basados en filtrado colaborativo.
# Al incluir el argumento rating_scale=(0.5, 5) al crear el objeto Reader,
# le estás diciendo a surprise que las calificaciones están en el rango de 0.5 a 5

# reader = Reader(rating_scale=(0.5, 5))
# df = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [ ]:
# El conjunto de entrenamiento (trainset) se crea con build_full_trainset() en Surprise.
# Contiene información vital como calificaciones, usuarios, ítems y la calificación media global.
# Este conjunto es esencial para entrenar modelos de recomendación personalizados.

# trainset = df.build_full_trainset()

# trainset.ur proporciona las calificaciones dadas por un usuario específico.
# Puedes explorar listas de usuarios e ítems con all_users() y all_items().
# trainset.global_mean da la calificación media global en el conjunto de entrenamiento.

# user_id = 264
# user_ratings = trainset.ur[user_id]
# print(user_ratings)

In [ ]:
# Se instancia un modelo de descomposición de valores singulares (SVD) utilizando SVD().
# Se evalúa el rendimiento del modelo utilizando la validación cruzada (cross_validate).

# svd = SVD()

# El modelo se ajusta y evalúa en cinco divisiones (cv=5) del conjunto de datos, utilizando
# métricas como Error Cuadrático Medio (RMSE) y Error Absoluto Medio (MAE).
# La evaluación proporciona información sobre la precisión y el rendimiento general del modelo en datos no vistos.
# El parámetro verbose=True muestra detalles adicionales durante el proceso de evaluación.
# Evaluación del modelo usando cross_validate

# cross_validate(svd, df, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# Se entrena el modelo SVD utilizando el conjunto de entrenamiento completo.

# svd.fit(trainset)

In [ ]:
# Dado el ID de la peícula, 215, el id del usuario 29801 y
# el valor de calificación real que se utilizará para evaluar
# la predicción. Si no se proporciona, la función devolverá
# solo la predicción estimada. Intenta predecir el valor.

# svd.predict(29801, 215, 5)

In [ ]:
def recommend_movies_svd(user_id, dataset, n_recommendations=5):
    """
    Recomienda películas no vistas para un usuario utilizando el algoritmo SVD.

    Parámetros:
    - user_id: ID del usuario para el cual se desean recomendaciones.
    - dataset: DataFrame con las calificaciones de los usuarios.
    - n_recommendations: Número de películas a recomendar.

    Retorna:
    - Lista de tuplas (movieId, predicted_rating) para las películas recomendadas.
    """

    # Configuración del lector y carga del conjunto de datos en Surprise
    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(dataset[['userId', 'movieId', 'rating']], reader)

    # División en conjunto de entrenamiento y prueba
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    # Inicialización del modelo SVD
    svd = SVD()

    # Entrenamiento del modelo en el conjunto de entrenamiento
    svd.fit(trainset)

    # Obtención de las películas no vistas por el usuario
    user_movies = dataset[dataset['userId'] == user_id]['movieId']
    unrated_movies = dataset[~dataset['movieId'].isin(user_movies)]['movieId'].unique()

    # Predicciones para las películas no vistas
    predictions = [(movie_id, svd.predict(user_id, movie_id).est) for movie_id in unrated_movies]

    # Ordenar las predicciones por rating estimado de mayor a menor
    recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)

    # Tomar las primeras n_recommendations películas recomendadas
    top_recommendations = recommendations[:n_recommendations]

    return top_recommendations


In [ ]:
def recommend_movies_svd(user_id, dataset, n_recommendations=10):
    """
    Recomienda películas no vistas para un usuario utilizando el algoritmo SVD.

    Parámetros:
    - user_id: ID del usuario para el cual se desean recomendaciones.
    - dataset: DataFrame con las calificaciones de los usuarios.
    - n_recommendations: Número de películas a recomendar.

    Retorna:
    - DataFrame con columnas 'title' y 'predicted_rating' para las películas recomendadas.
    """

    # Configuración del lector y carga del conjunto de datos en Surprise
    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(dataset[['userId', 'movieId', 'rating']], reader)

    # División en conjunto de entrenamiento y prueba
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    # Inicialización del modelo SVD
    svd = SVD()

    # Entrenamiento del modelo en el conjunto de entrenamiento
    svd.fit(trainset)

    # Obtención de las películas no vistas por el usuario
    user_movies = dataset[dataset['userId'] == user_id]['movieId']
    unrated_movies = dataset[~dataset['movieId'].isin(user_movies)]['movieId'].unique()

    # Predicciones para las películas no vistas
    predictions = [(movie_id, svd.predict(user_id, movie_id).est) for movie_id in unrated_movies]

    # Ordenar las predicciones por rating estimado de mayor a menor
    recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:n_recommendations]

    # Obtener los títulos de las películas recomendadas
    recommended_movies = [(dataset[dataset['movieId'] == movie_id]['title'].iloc[0], rating) for movie_id, rating in recommendations]

    # Crear un DataFrame a partir de las recomendaciones
    df_recommended_movies = pd.DataFrame(recommended_movies, columns=['title', 'predicted_rating'])

    return df_recommended_movies

In [ ]:
# Supongamos que tienes tu DataFrame de calificaciones 'ratings'
# con columnas 'userId', 'movieId', y 'rating'.

# Define el ID del usuario para el cual quieres hacer recomendaciones
user_id_to_recommend = 264

# Define el número de películas que quieres recomendar
num_recommendations = 10

# Filtra las calificaciones del usuario específico
user_ratings = df[df['userId'] == user_id_to_recommend]
# Obtiene las películas vistas por el usuario sin duplicados
movies_watched = user_ratings[['title', 'rating']].drop_duplicates()
# Ordena las películas por calificación de mayor a menor
movies_watched_sorted = movies_watched.sort_values(by='rating', ascending=False)
# Muestra el DataFrame con los títulos de las películas vistas y las calificaciones
print("Películas vistas por el usuario {}: \n{}".format(user_id_to_recommend, movies_watched_sorted))

# Llama a la función para obtener las recomendaciones
recommendations = recommend_movies_svd(user_id_to_recommend, df, num_recommendations)

# Imprime las recomendaciones
print("Recomendaciones para el usuario {}".format(user_id_to_recommend))
recommendations.head(10)


---------------------------------------------------------


In [ ]:
# # Eliminar filas duplicadas
# # Eliminar filas duplicadas basadas en las columnas 'userId' y 'movieId'
# df_remove_duplicates_rows = df.copy()
# df_remove_duplicates_rows = df_remove_duplicates_rows.drop_duplicates(subset=['userId', 'movieId'])
# pivot_table_result_rm_duplic_rw = df_remove_duplicates_rows.pivot_table(index='userId', columns='title', values='rating')
# pivot_table_result_rm_duplic_rw.fillna(0, inplace=True)
# pivot_table_result_rm_duplic_rw.head()